In [14]:
import numpy as np
import gzip
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
# from ids import *
from block_rect_maxvol import *

In [2]:
fn = 'Channel120.txt.gz'

In [46]:
def LoadRawData(fn):
    if fn[-3:] == '.gz':
        op = gzip.open
    else:
        op = open
        
    with op(fn, 'r') as f:
        data = [line.strip().split("\t") for line in f.readlines()]
    return data

def ProssData(d):
    ans = {}
    nums = {}
    for l in d:
        key = tuple(int(i) for i in l[:3])
        num = int(l[3])
        df = np.array([float(i) for i in l[4::2]]) + \
              np.array([float(i) for i in l[5::2]])*1j
                                 
        if key in ans:
            ans[key].append(df)
            nums[key].append(num)
        else:
            ans[key] = [df]
            nums[key] = [num]

    for key in ans:
        # print  ans[key]
        ans[key] = np.array(ans[key])
        # print  ans[key]
        
    return ans, np.array(nums)

def EqualClusters(data, num_clusters, ToReIdx=False):
    # See https://stats.stackexchange.com/questions/8744/clustering-procedure-where-each-cluster-has-an-equal-number-of-points
    n = data.shape[0]
    m = n // num_clusters
    if n != m*num_clusters:
        print "Data will be cut"
        n = m*num_clusters
        data = data[:n]
        
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(data)
    dst = kmeans.fit_transform(data)
    dst2 = dst**2
    dst = np.tile(dst2, m)
    rr, cl = linear_sum_assignment(dst)
    assert(np.all(rr == np.arange(n)))
    
    if ToReIdx:
        cl = ReIdx(cl, num_clusters)
        
    return cl 

def ReIdx(idx, m):
    return np.hstack((idx[i::m] for i in np.arange(m)))

    

In [6]:
d = LoadRawData(fn)
res, nums = ProssData(d)

In [7]:
# Check first matrices data sizes
cnt = 0
for i in res:
    print i, res[i].shape
    cnt += 1
    if cnt > 10:
        break

(3, 26, 3) (173, 64)
(1, 38, 2) (144, 64)
(5, 48, 3) (171, 64)
(2, 33, 1) (161, 64)
(2, 14, 2) (145, 64)
(2, 8, 2) (145, 64)
(2, 25, 3) (174, 64)
(4, 39, 3) (173, 64)
(2, 19, 3) (174, 64)
(2, 40, 1) (161, 64)
(1, 48, 1) (162, 64)


In [50]:
nder = 5
cnt = 0
num_clusters = 10
for i in res:
    A = res[i]
    m, sz = A.shape
    num_cl = m//nder
    print i, m, sz, num_cl
    idx = EqualClusters(A, num_clusters)
    bm = rect_block_maxvol(res[i], nder, Kmax = sz, max_iters=100, rect_tol = 0.05, tol = 0.0, debug = False, ext_debug = False)
    print bm
    
    cnt += 1
    if cnt > 10:
        break


(3, 26, 3) 173 64 34
Data will be cut


/home/gleb/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


LinAlgError: Last 2 dimensions of the array must be square

In [47]:
X = np.array([[1, 2], [1, 4], [1, 0],
               [4, 2], [4, 4], [4, 0]])

kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
print kmeans.labels_

print EqualClusters(X, 3, ToReIdx=False)
print EqualClusters(X, 3, ToReIdx=True)


[0 0 0 1 2 1]
[3 2 0 1 5 4]
[3 1 2 5 0 4]
